In [1]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [69]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from config import gkey
import gmaps
import gmaps.geojson_geometries
from matplotlib.cm import viridis, inferno, plasma
from matplotlib.colors import to_hex
gmaps.configure(api_key= gkey)

In [3]:
rds_connection_string = "postgres:postgres@localhost:5432/COVID_df"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [4]:
happy_df = pd.read_sql_query('select * from happiness_table', con=engine)

In [5]:
happy_df.head()

,country,ranking,score,social_support,freedom_index,corruption_index
0,Finland,1,7.769,1.587,0.596,0.393
1,Denmark,2,7.600,1.573,0.592,0.410
2,Norway,3,7.554,1.582,0.603,0.341
3,Iceland,4,7.494,1.624,0.591,0.118
4,Netherlands,5,7.488,1.522,0.557,0.298


In [6]:
covid_df = pd.read_sql_query('select * from covid_cases', con=engine)

In [7]:
covid_df.head()

,country_region,population,totalcases,totaldeaths,totaltests
0,USA,330854064.0,1881205.0,108059.0,18603174.0
1,Brazil,212442762.0,558237.0,31309.0,930013.0
2,Russia,145929848.0,423741.0,5037.0,11151622.0
3,Spain,46753443.0,287012.0,27127.0,4063843.0
4,UK,67858826.0,277985.0,39369.0,4615146.0


In [8]:
country_df = pd.read_sql_query('select * from countrycode_table', con=engine)

In [9]:
country_df.head()

,iso_a3,country,continent,subregion,country_code,continent_code,subregion_code
0,AFG,Afghanistan,Asia,Southern Asia,4,142.0,34.0
1,ALA,Åland Islands,Europe,Northern Europe,248,150.0,154.0
2,ALB,Albania,Europe,Southern Europe,8,150.0,39.0
3,DZA,Algeria,Africa,Northern Africa,12,2.0,15.0
4,ASM,American Samoa,Oceania,Polynesia,16,9.0,61.0


In [34]:
merged_df = pd.merge(happy_df, country_df, how='inner', left_on='country', right_on='country')
merged_df.head()

,country,ranking,score,social_support,freedom_index,corruption_index,iso_a3,continent,subregion,country_code,continent_code,subregion_code
0,Finland,1,7.769,1.587,0.596,0.393,FIN,Europe,Northern Europe,246,150.0,154.0
1,Denmark,2,7.600,1.573,0.592,0.410,DNK,Europe,Northern Europe,208,150.0,154.0
2,Norway,3,7.554,1.582,0.603,0.341,NOR,Europe,Northern Europe,578,150.0,154.0
3,Iceland,4,7.494,1.624,0.591,0.118,ISL,Europe,Northern Europe,352,150.0,154.0
4,Netherlands,5,7.488,1.522,0.557,0.298,NLD,Europe,Western Europe,528,150.0,155.0


In [35]:
merged_df.set_index("country", inplace=True)
merged_df.head()

,ranking,score,social_support,freedom_index,corruption_index,iso_a3,continent,subregion,country_code,continent_code,subregion_code
country,,,,,,,,,,,
Finland,1,7.769,1.587,0.596,0.393,FIN,Europe,Northern Europe,246,150.0,154.0
Denmark,2,7.600,1.573,0.592,0.410,DNK,Europe,Northern Europe,208,150.0,154.0
Norway,3,7.554,1.582,0.603,0.341,NOR,Europe,Northern Europe,578,150.0,154.0
Iceland,4,7.494,1.624,0.591,0.118,ISL,Europe,Northern Europe,352,150.0,154.0
Netherlands,5,7.488,1.522,0.557,0.298,NLD,Europe,Western Europe,528,150.0,155.0


### Add GeoJSON layer

In [10]:
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [11]:
fig = gmaps.figure(layout=figure_layout)

In [12]:
countries_geojson = gmaps.geojson_geometries.load_geometry('countries')

In [13]:
gini_layer = gmaps.geojson_layer(countries_geojson)

In [14]:
fig.add_layer(gini_layer)

In [15]:
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [106]:
def calculate_color(input_data):
    # input_data is between 0 and 1
    # invert gini so that high inequality gives dark color
    inverse_data = 1.0 - input_data

    # transform the gini coefficient to a matplotlib color
#     mpl_color = viridis(inverse_data)
#     mpl_color = plasma(inverse_data)
    mpl_color = inferno(inverse_data)
    print(mpl_color)
    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

In [107]:
a_data = merged_df["freedom_index"]
country_names = merged_df.index
print(country_names)
country_codes = list(merged_df["iso_a3"])
country_codes
b = merged_df.loc[merged_df.iso_a3=="FIN"]["freedom_index"]
print(b)
c = merged_df.loc[merged_df.index=="Finland"]["freedom_index"]
print(c)

Index(['Finland', 'Denmark', 'Norway', 'Iceland', 'Netherlands', 'Switzerland',
       'Sweden', 'New Zealand', 'Canada', 'Austria',
       ...
       'Haiti', 'Botswana', 'Syria', 'Malawi', 'Yemen', 'Rwanda', 'Tanzania',
       'Afghanistan', 'Central African Republic', 'South Sudan'],
      dtype='object', name='country', length=146)
country
Finland    0.596
Name: freedom_index, dtype: float64
country
Finland    0.596
Name: freedom_index, dtype: float64


In [108]:
colors=[]
for feature in countries_geojson['features']:
    country_name = feature['properties']['name']
    country_code = feature['properties']['ISO_A3']
    print(country_name)
    try:
        if country_name in country_names:
            print(country_name)
            input_value = merged_df.loc[merged_df.index==country_name]["freedom_index"][0]
            print(input_value)
        elif country_code in country_codes:
            input_value = merged_df.loc[merged_df.iso_a3==country_code]["freedom_index"][0]
            print(country_code)
        else: input_value = 0
        color = calculate_color(input_value)
    except KeyError:
        # no GINI for that country: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

Afghanistan
Afghanistan
0.0
(0.988362, 0.998364, 0.644924, 1.0)
Angola
(0.988362, 0.998364, 0.644924, 1.0)
Albania
Albania
0.38299999999999995
(0.882188, 0.337287, 0.207628, 1.0)
Aland
(0.988362, 0.998364, 0.644924, 1.0)
Andorra
(0.988362, 0.998364, 0.644924, 1.0)
United Arab Emirates
United Arab Emirates
0.598
(0.578304, 0.148039, 0.404411, 1.0)
Argentina
Argentina
0.47100000000000003
(0.775059, 0.239667, 0.303526, 1.0)
Armenia
Armenia
0.28300000000000003
(0.963387, 0.495462, 0.079073, 1.0)
American Samoa
(0.988362, 0.998364, 0.644924, 1.0)
Antarctica
(0.988362, 0.998364, 0.644924, 1.0)
French Southern and Antarctic Lands
(0.988362, 0.998364, 0.644924, 1.0)
Antigua and Barbuda
(0.988362, 0.998364, 0.644924, 1.0)
Australia
Australia
0.557
(0.64626, 0.173914, 0.378359, 1.0)
Austria
Austria
0.532
(0.682656, 0.189501, 0.360757, 1.0)
Azerbaijan
Azerbaijan
0.35100000000000003
(0.916462, 0.387481, 0.164924, 1.0)
Burundi
Burundi
0.22
(0.985315, 0.608422, 0.024202, 1.0)
Belgium
Belgium
0.473
(

In [101]:
fig = gmaps.figure()
gini_layer = gmaps.geojson_layer(
    countries_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.9)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Using Bins and color

In [102]:
merged_df.ranking.max()

156

In [117]:
bins = [1, 11, 21, 31, 41, 51, 156]
color_code_label = [0,1,2,3,4,5]
color_code = [(191/255,45/255,45/255,1.0),(227/255,109/255,109/255,1.0),(241/255,135/255,135/255,1.0),(245/255,173/255,173/255,1.0),(1.0, 201/255, 201/255,1.0),(1.0,226/255,226/255,1.0)]
merged_df["color_code"] = pd.cut(merged_df["ranking"], bins, labels=color_code_label, include_lowest=True)

In [118]:
merged_df

,ranking,score,social_support,freedom_index,corruption_index,iso_a3,continent,subregion,country_code,continent_code,subregion_code,color_code
country,,,,,,,,,,,,
Finland,1,7.769,1.587,0.596,0.393,FIN,Europe,Northern Europe,246,150.0,154.0,0
Denmark,2,7.600,1.573,0.592,0.410,DNK,Europe,Northern Europe,208,150.0,154.0,0
Norway,3,7.554,1.582,0.603,0.341,NOR,Europe,Northern Europe,578,150.0,154.0,0
Iceland,4,7.494,1.624,0.591,0.118,ISL,Europe,Northern Europe,352,150.0,154.0,0
Netherlands,5,7.488,1.522,0.557,0.298,NLD,Europe,Western Europe,528,150.0,155.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
Rwanda,152,3.334,0.711,0.555,0.411,RWA,Africa,Sub-Saharan Africa,646,2.0,202.0,5
Tanzania,153,3.231,0.885,0.417,0.147,TZA,Africa,Sub-Saharan Africa,834,2.0,202.0,5
Afghanistan,154,3.203,0.517,0.000,0.025,AFG,Asia,Southern Asia,4,142.0,34.0,5


In [119]:
colors=[]
for feature in countries_geojson['features']:
    country_name = feature['properties']['name']
    country_code = feature['properties']['ISO_A3']
    print(country_name)
    try:
        if country_name in country_names:
            print(country_name)
            input_value = merged_df.loc[merged_df.index==country_name]["color_code"][0]
            print(input_value)
        elif country_code in country_codes:
            input_value = merged_df.loc[merged_df.iso_a3==country_code]["color_code"][0]
            print(country_code)
        else: input_value = 5
        color = to_hex(color_code[input_value], keep_alpha=False)
    except KeyError:
        # no GINI for that country: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

Afghanistan
Afghanistan
5
Angola
Albania
Albania
5
Aland
Andorra
United Arab Emirates
United Arab Emirates
1
Argentina
Argentina
4
Armenia
Armenia
5
American Samoa
Antarctica
French Southern and Antarctic Lands
Antigua and Barbuda
Australia
Australia
0
Austria
Austria
0
Azerbaijan
Azerbaijan
5
Burundi
Burundi
5
Belgium
Belgium
1
Benin
Benin
5
Burkina Faso
Burkina Faso
5
Bangladesh
Bangladesh
5
Bulgaria
Bulgaria
5
Bahrain
Bahrain
3
The Bahamas
Bosnia and Herzegovina
Belarus
Belarus
5
Belize
Bolivia
Bolivia
5
Brazil
Brazil
3
Barbados
Brunei
Bhutan
Bhutan
5
Botswana
Botswana
5
Central African Republic
Central African Republic
5
Canada
Canada
0
Switzerland
Switzerland
0
Chile
Chile
2
China
China
5
Ivory Coast
Cameroon
Cameroon
5
Cyprus No Mans Area
Democratic Republic of the Congo
Republic of Congo
Colombia
Colombia
4
Comoros
Comoros
5
Cape Verde
Costa Rica
Costa Rica
1
Cuba
Curaçao
Cayman Islands
Northern Cyprus
Cyprus
Cyprus
4
Czech Republic
Czech Republic
1
Germany
Germany
1
Djibouti
Do

In [120]:
fig = gmaps.figure()
gini_layer = gmaps.geojson_layer(
    countries_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.9)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(height='420px'))